# Sample Sentiment Analysis

In [1]:
from pypsenti.service.sentiment import SentimentAnalysis, SentimentConnection, Document, add_logger
from sklearn import metrics
import socket
user_name = socket.gethostname()
connection = SentimentConnection(host='192.168.0.70', port=7044, client_id=user_name)
add_logger()

###  Supported Custom Domains

In [2]:
for domain in connection.supported_domains:
    print(domain)

TwitterMarket
TwitterTrump
market


# Standard Lexicon Sentiment Analysis

In [3]:
%%time
amazon_reviews = ['I love this hello kitty decal! I like that the bow is pink instead of red. Only bad thing is that after putting it on the window theres a few air bubbles, but that most likely my fault. Shipped fast too.',
                  'I bought this for my 3 yr old daughter when I took it out the pack it had a bad oder, cute but very cheap material easy to ripe.  When I tried it on her it was to big, but of course she liked it so I kept it. I dressed her up in it and she looked cute.']

analysis = SentimentAnalysis(connection)

for result in analysis.detect_sentiment_text(amazon_reviews):
    print(f'{result["Stars"]:1.2f}')

2019-12-02 00:11:20,793 - pypsenti.service - INFO - Setting up connection to 192.168.0.70:7044 for hp-z8
2019-12-02 00:11:20,795 - pypsenti.service - DEBUG - Opening Connection...
2019-12-02 00:11:20,796 - pypsenti.service - DEBUG - Sending CONNECT (u0, p0, wr0, wq0, wf0, c1, k60) client_id=b'hp-z8'
2019-12-02 00:11:20,798 - pypsenti.service - DEBUG - wait_connected
2019-12-02 00:11:20,819 - pypsenti.service - DEBUG - Received CONNACK (0, 0)
2019-12-02 00:11:20,822 - pypsenti.service - DEBUG - on_connect
2019-12-02 00:11:21,000 - pypsenti.service - DEBUG - subscribing to topicSentiment/Result/hp-z8
2019-12-02 00:11:21,000 - pypsenti.service - DEBUG - Sending SUBSCRIBE (d0, m1) [(b'Sentiment/Result/hp-z8', 1)]
2019-12-02 00:11:21,001 - pypsenti.service - DEBUG - subscribe returned (0, 1)
2019-12-02 00:11:21,002 - pypsenti.service - DEBUG - wait_subscribed
2019-12-02 00:11:21,003 - pypsenti.service - DEBUG - Received SUBACK
2019-12-02 00:11:21,004 - pypsenti.service - DEBUG - on_subscrib

3.58
3.68
Wall time: 2.23 s


In [4]:
analysis = SentimentAnalysis(SentimentConnection(user_name), model='Test')

for result in analysis.detect_sentiment(documents):
    print(result['Stars'])

NameError: name 'documents' is not defined

In [5]:
%%time
id = 0 
all_amazon_documents = []
true_document_class = {}
with open('../data/amazon/positive.txt', "r", encoding='utf8') as reader:
    for line in reader:
        doc = Document(id, line)
        doc.isPositive = True
        all_amazon_documents.append(doc)
        true_document_class[doc.id] = True
        id += 1
    
with open('../data/amazon/negative.txt', "r", encoding='utf8') as reader:
    for line in reader:
        doc = Document(id, line)    
        doc.isPositive = False
        all_amazon_documents.append(doc)
        true_document_class[doc.id] = False
        id += 1
    
detected_document_class = {}
print(f'Processig {len(all_amazon_documents)} documents...')
analysis = SentimentAnalysis(connection)
for result in analysis.detect_sentiment(all_amazon_documents):
    stars = result['Stars']    
    id  = result['Id']
    detected_document_class[id] = stars is not None and stars > 3
    
print(f'Total processed documents: {len(detected_document_class)}')

test_y = [true_document_class[document.id] for document in all_amazon_documents]
result_y = [detected_document_class[document.id] for document in all_amazon_documents]
vacc = metrics.accuracy_score(test_y, result_y)
print(f'Accuracy: {vacc}')

#

Processig 2000 documents...
Total processed documents: 2000
Accuracy: 0.776
Wall time: 11.4 s


In [6]:
%%time
connection.save_documents('Amazon', all_amazon_documents)
analysis = SentimentAnalysis(connection)

Wall time: 3.14 s


In [7]:
%%time
analysis.train('Amazon')

Wall time: 8.99 s


In [8]:
%%time
analysis = SentimentAnalysis(connection, model='Amazon')
for result in analysis.detect_sentiment(all_amazon_documents):
    stars = result['Stars']    
    id  = result['Id']
    detected_document_class[id] = stars is not None and stars > 3
    
print(f'Total processed documents: {len(detected_document_class)}')

test_y = [true_document_class[document.id] for document in all_amazon_documents]
result_y = [detected_document_class[document.id] for document in all_amazon_documents]
vacc = metrics.accuracy_score(test_y, result_y)
print(f'Accuracy: {vacc}')

Total processed documents: 2000
Accuracy: 0.8895
Wall time: 8.01 s
